Support Vector Machine (SVM) を実行してみる。 [参考ドキュメント](http://scikit-learn.org/stable/modules/svm.html)

In [1]:
import numpy as np
import pandas as pd
import sys

from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR


print(sys.version_info)

sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)


### 1. データの準備

Toyデータを用意する。  
[make_regression](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html)関数を使うと自身が望むような回帰分析の検証のためのサンプルデータを用意することができる。  
今回は`サンプル数=1000`, `説明変数の数=10`, そして再現性を担保するために`random_state=0`, `shuffle=False`とした。

In [2]:
X, y = make_regression(n_samples=1000, n_features=10,
                                                random_state=0, shuffle=False)
X = pd.DataFrame(X)

In [3]:
X.head()

,0,1,2,3,4,5,6,7,8,9
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599
1,0.144044,1.454274,0.761038,0.121675,0.443863,0.333674,1.494079,-0.205158,0.313068,-0.854096
2,-2.552990,0.653619,0.864436,-0.742165,2.269755,-1.454366,0.045759,-0.187184,1.532779,1.469359
3,0.154947,0.378163,-0.887786,-1.980796,-0.347912,0.156349,1.230291,1.202380,-0.387327,-0.302303
4,-1.048553,-1.420018,-1.706270,1.950775,-0.509652,-0.438074,-1.252795,0.777490,-1.613898,-0.212740


In [4]:
y[:5]

array([ 300.2064792 ,  194.27686437,   13.25754564,  -24.6027897 ,
        -98.76061926])

サンプルサイズと説明変数の数は`shape`に保存されている。

In [5]:
X.shape

(1000, 10)

### 2. Support Vector Machine Regression (SVR)

#### 2.1 機械学習

`kernel='rbf'`, `C=1.0`, `epsilon=0.2`として[SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)を行ってみる。 

In [6]:
regr = SVR(kernel='rbf', C=1.0, epsilon=0.2)
regr.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

#### 2.2 予測および性能評価

予測値を求めるには`predict`メソッドを用いる。

In [7]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 26.94403357,  35.96696189,  -3.57808668, -10.28559151, -18.71320019])

[mean_squared_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)関数を用いてMean Square Error (MSE) を求める。

In [8]:
mean_squared_error(y, y_predicted)

13634.495133888204

また`score`メソッドを用いることで実測値と予測値の間の決定係数(r<sup>2</sup>)を求めることができる。

In [9]:
regr.score(X, y)

0.28662093366953867

かなり決定係数の値が小さいことがわかる。  
ここから`kernel`, `C`, `epsilon`, `gamma`, `degree`をグリッドサーチすることによってチューニングして値を大きくしてみる。

#### 2.3 グリッドサーチによるパラメータチューニング

パラメータチューニングは何度も反復して計算するため、計算コストが非常に高くなる場合があります。<br>
Jupyter Notebookでは`%%time`とセルの最初に書くことで計算にかかった時間が表示されます。  
これ以降の計算は私のパソコンでどれぐらい計算に時間がかかったか目安として書いていますので、  
そちらを参考に計算するか判断して実行してみてください。

##### 2.3.1 チューニングするパラメータの定義

SVRのパラメータは最大で４つあり個々に変化させて調べるのは大変なので[GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)クラスを用いる。  
GridSearchCVは今回のグリッドサーチのような網羅的な探索が必要な時に使う関数である。  

まずはチューニングするパラメータの取り得るグリッドの幅を決める。
以下のパラメータでは
- `kernel`は`poly`, `rbf`, `sigmoid`の計3つ
- `C`は`1/4`, `1/2`, `0`, `2`, `4`の計5つ  

用いるので、計3 × 5 = 15パターンのSVRのパラメータでモデルを構築することになる。

In [10]:
parameters = {
    'kernel': ('poly', 'rbf', 'sigmoid'),
    'C': (2**-2, 2**-1, 2**0, 2**1, 2**2),
}

##### 2.3.2 グリッドサーチを行う

今回は5-fold cross validationを行ってチューニングするようにする。  
したがって計15パターン × 5fold = 75回モデルを構築することになる。

In [11]:
%%time
svr = SVR()
regr = GridSearchCV(svr, parameters, cv=5)
regr.fit(X, y)

CPU times: user 5.3 s, sys: 90 ms, total: 5.39 s
Wall time: 5.4 s


約5秒で計算が終了した。  
`best_estimator_`に最も性能が良い時のモデルとそのモデルのパラメータの値が保存されている。

In [12]:
print(f'Best Estimator:\n{regr.best_estimator_}\n')

Best Estimator:
SVR(C=4, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False)



今回のデータでは、`kernel=sigmoid`, `C=4`のときが最も性能が良い。  
ただし今回の例では`C=4`はパラメータ候補の一番端であり、`C=8`, `C=16`としていくとより良いスコアを得られる可能性はあるため、  
さらに性能を良くしたい時にはパラメータ候補を増やしていく必要がある。

続けて2.2の結果と比較するために実測値`y`と予測値の間の決定係数(r<sup>2</sup>)を`score`を用いて求めてみる。

In [13]:
regr.best_estimator_.score(X, y)

0.9944363691782484

2.2で求めたチューニングをまったくしていない時の値である約0.29と比べるとはるかに良くなっているのがわかる。

また`cv_results_`にチューニングのすべての結果が保存されている。以下に一部を表示してみる。

In [14]:
df = pd.DataFrame(regr.cv_results_)
df.head()

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_kernel,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.031598,0.002819,0.054519,0.062209,0.25,poly,"{'C': 0.25, 'kernel': 'poly'}",15,0.057751,0.062491,...,0.055802,0.061496,0.062406,0.061906,0.040311,0.061782,0.001807,0.000220,0.007476,0.000664
1,0.044833,0.006114,0.056380,0.061911,0.25,rbf,"{'C': 0.25, 'kernel': 'rbf'}",14,0.059110,0.062045,...,0.057075,0.061561,0.059550,0.063623,0.045074,0.060752,0.001592,0.000459,0.005796,0.000952
2,0.055059,0.007849,0.170023,0.174255,0.25,sigmoid,"{'C': 0.25, 'kernel': 'sigmoid'}",11,0.172404,0.174626,...,0.169084,0.173071,0.175603,0.176852,0.157821,0.172863,0.008022,0.000858,0.006533,0.001441
3,0.036493,0.003507,0.111106,0.122362,0.5,poly,"{'C': 0.5, 'kernel': 'poly'}",13,0.113705,0.122901,...,0.112197,0.120939,0.122731,0.121560,0.095677,0.122082,0.002870,0.000449,0.008731,0.001175
4,0.046956,0.006114,0.114682,0.121958,0.5,rbf,"{'C': 0.5, 'kernel': 'rbf'}",12,0.116320,0.122209,...,0.114788,0.121309,0.117142,0.124943,0.104525,0.120262,0.002765,0.000227,0.005429,0.001617


#### 2.3.3 さらにグリッドサーチを行う

グリッドサーチをするパラメータの数を増やしてチューニングを行う。  
以下のパラメータでは
- `kernel`は`poly`, `rbf`, `sigmoid`の計3つ
- `C`は`1/4`, `1/2`, `0`, `2`, `4`の計5つ  
- `epsilon`は`1/4`, `1/2`, `0`, `2`, `4`の計5つ  
用いるので、計3 × 5 × 5 = 75パターンのSVRのパラメータでモデルを構築することになる。

In [15]:
parameters = {
    'kernel': ('poly', 'rbf', 'sigmoid'),
    'C': (2**-2, 2**-1, 2**0, 2**1, 2**2),
    'epsilon': (2**-2, 2**-1, 2**0, 2**1, 2**2),
}

今回も5-fold cross validationを行ってチューニングするようにする。  
したがって計75パターン × 5fold = 375回モデルを構築することになる。

In [16]:
%%time
svr = SVR()
regr = GridSearchCV(svr, parameters, cv=5)
regr.fit(X, y)

CPU times: user 24.5 s, sys: 660 ms, total: 25.2 s
Wall time: 25.2 s


約30秒で計算が終了した。  
`best_estimator_`に最も性能が良い時のモデルとそのモデルのパラメータの値が保存されている。

In [17]:
print(f'Best Estimator:\n{regr.best_estimator_}\n')

Best Estimator:
SVR(C=4, cache_size=200, coef0=0.0, degree=3, epsilon=2, gamma='auto',
  kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False)



今回のデータでは、`kernel=sigmoid`, `C=4`, `epsilon=2`のときが最も性能が良い。  
ただし2.3.2同様、今回の例では`C=4`はパラメータ候補の一番端であり、`C=8`, `C=16`としていくとより良いスコアを得られる可能性はあるため、  
さらに性能を良くしたい時にはパラメータ候補を増やしていく必要がある。

続けて2.2の結果と比較するために実測値`y`と予測値の間の決定係数(r<sup>2</sup>)を`score`を用いて求めてみる。

In [18]:
regr.best_estimator_.score(X, y)

0.99490138026625841

2.2はもちろん2.3.2で求めた値よりも(2.3.2とはごくわずかであるが)良くなっているのがわかる。

#### 2.3.4 さらにさらにグリッドサーチを行う

さらにグリッドサーチをするパラメータの数を増やしてチューニングを行う。  
以下のパラメータでは
- `kernel`は`poly`, `rbf`, `sigmoid`の計3つ
- `C`は`1/4`, `1/2`, `0`, `2`, `4`の計5つ  
- `epsilon`は`1/4`, `1/2`, `0`, `2`, `4`の計5つ  
- `gamma`は`2の-14乗`, `2の-12乗`, `2の-10乗`, `2の-8乗`, `2の-6乗`の計5つ  
用いるので、計3 × 5 × 5 × 5 = 375パターンのSVRのパラメータでモデルを構築することになる。

In [19]:
parameters = {
    'kernel': ('poly', 'rbf', 'sigmoid'),
    'C': (2**-2, 2**-1, 2**0, 2**1, 2**2),
    'epsilon': (2**-2, 2**-1, 2**0, 2**1, 2**2),
    'gamma': (2**-14, 2**-12, 2**-10, 2**-8, 2**-6),
}

今回も5-fold cross validationを行ってチューニングするようにする。  
したがって計375パターン × 5fold = 1875回モデルを構築することになる。

In [20]:
%%time
svr = SVR()
regr = GridSearchCV(svr, parameters, cv=5)
regr.fit(X, y)

CPU times: user 1min 55s, sys: 2.52 s, total: 1min 58s
Wall time: 1min 58s


約2分で計算が終了した。  
`best_estimator_`に最も性能が良い時のモデルとそのモデルのパラメータの値が保存されている。

In [21]:
print(f'Best Estimator:\n{regr.best_estimator_}\n')

Best Estimator:
SVR(C=4, cache_size=200, coef0=0.0, degree=3, epsilon=0.25, gamma=0.015625,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)



今回のデータでは、`kernel=rbf`, `C=4`, `epsilon=0.25`, `gamma=0.015625`のときが最も性能が良い。  
この時の2.2の結果と比較するために実測値`y`と予測値の間の決定係数(r<sup>2</sup>)を`score`を用いて求めてみる。

In [22]:
regr.best_estimator_.score(X, y)

0.69281542406271479

2.2よりはよくなったものの2.3.2, 2.3.3より大幅に悪くなってしまった。  
これはデフォルトでは `gamma=1/説明変数の数` という値を使うが、今回は計算コストを下げるために`gamma`の値を小さくして行った。  
その結果、局所最適解に陥り、今回の候補のパラメータの中では最も良いもののもっとはるかに良いパラメータの値が存在することになってしまった。  

このようなことにもならないように適切なパラメータの候補を設定するのが望ましい。  
なお `gamma` の値は大きくすると計算コストが大きくなる傾向があるので伸長に増やしていく必要がある。  
私も今回の例でなかなか計算が終わらず途中で何度も強制終了するということを繰り返した。もうこれ以上したくない。